In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options


import datetime
import time
import csv
import openpyxl
from openpyxl.styles import PatternFill
import mysql.connector

### CSV
#### Funciones para guardar / cargar 

In [7]:
def guardarCSV(ruta:str, listaProductos):

    with open(ruta, mode='w', newline='') as productos_csv:
        writer = csv.writer(productos_csv)
        for Supermercado in listaProductos:
            writer.writerow(Supermercado)




def cargarCSV(ruta:str):
    lista_compra = []
    with open(ruta,'r',encoding='utf-8', newline='\n') as csvfile:
        productos = csv.reader(csvfile, delimiter=',')
        for product in productos:
            lista_compra.append(product)
    
    return lista_compra

### EXCEL
#### Funciones para guardar / cargar 

In [12]:
from openpyxl import Workbook


def GuardarExcel(ruta:str, listaProductos, listaPrecios):

    wb = Workbook()
    ws = wb.active
    ws.title = "Productos"

    
    ws['A1'] = "Productos"
    ws['B1'] = "Carrefour"
    ws['C1'] = "Ahorra Mas"
    ws['D1'] = "Dia"
    
    for i in range(len(listaProductos)):
        ws[f'A{i+2}'] = listaProductos[i][0]
        ws[f'B{i+2}'] = listaPrecios[i][0][1]
        ws[f'C{i+2}'] = listaPrecios[i][1][1]
        ws[f'D{i+2}'] = listaPrecios[i][2][1]

        #Comprobar Precio
        if ComprobarPrecio(listaPrecios[i]) == 0:
            ws[f'B{i+2}'].fill =  PatternFill(start_color='DDF7E3', end_color='DDF7E3', fill_type='solid')

        elif ComprobarPrecio(listaPrecios[i]) == 1:
            ws[f'C{i+2}'].fill =  PatternFill(start_color='DDF7E3', end_color='DDF7E3', fill_type='solid')

        else: 
            ws[f'D{i+2}'].fill =  PatternFill(start_color='DDF7E3', end_color='DDF7E3', fill_type='solid')



    wb.save(ruta)


#Funcion que primeramente comprobara si hay algun producto que no exista en su pagina, todo esto para todas las tiendas.
#Una vez compruebe que todos los productos existen comprobara que producto tiene un precio menor retornando el numero de celda
#que debera pintar.
def ComprobarPrecio(lista):
    if lista[0][1] == 'NO EXISTE EL PRODUCTO':
        if lista[1][1] == 'NO EXISTE EL PRODUCTO':
            return 2
        elif lista[2][1] == 'NO EXISTE EL PRODUCTO':
            return 1
        elif lista[2][1] >  lista[1][1]:
            return 1
        elif lista[2][1] <  lista[1][1]:
            return 2
        
    if lista[1][1] == 'NO EXISTE EL PRODUCTO':
        if lista[0][1] == 'NO EXISTE EL PRODUCTO':
            return 2
        elif lista[2][1] == 'NO EXISTE EL PRODUCTO':
            return 0
        elif lista[0][1] >  lista[2][1]:
            return 2
        elif lista[0][1] <  lista[2][1]:
            return 0

    if lista[2][1] == 'NO EXISTE EL PRODUCTO':
        if lista[0][1] == 'NO EXISTE EL PRODUCTO':
            return 1
        elif lista[1][1] == 'NO EXISTE EL PRODUCTO':
            return 0
        elif lista[0][1] >  lista[1][1]:
            return 1
        elif lista[0][1] <  lista[1][1]:
            return 0

    if lista[0][1] > lista[1][1] and lista[1][1] > lista[2][1]:
        return 2
    elif lista[1][1] > lista[0][1] and lista[2][1] > lista[0][1]:
        return 0
    else: return 1

In [20]:
class Search:
    product_name = ""

    def __init__(self, product_name: str, driver, cookies:bool):
        self.product_name = product_name
        self.driver = driver
        self.wait1 = WebDriverWait(self.driver, 10)
        self.cookies = cookies
    
    # Función que buscara el producto pasado a la clase en la web del carrefour
    def CarrefourProduct(self):
        self.driver.get(
            'https://www.carrefour.es/?gclid=Cj0KCQiAgaGgBhC8ARIsAAAyLfHJZAkh3PZU6zJ6jhoAhxnOvsCcPahmWyNHF4xUuQiT2F9gBk3rdloaAkgGEALw_wcB&gclsrc=aw.ds')

        # Verificara si hemos aceptado las cookies anteriormente de esta página
        if not self.cookies:
            btn_cookie = self.wait1.until(EC.presence_of_element_located((By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')))
            btn_cookie.click()

        # A la hora de buscar hay dos inputs diferentes el segundo se crea cuando clickamos el primero
        input_text_search_parent = self.obtenerXpath('//*[@id="search-input"]')
        input_text_search_parent.click()

        # Introducimos el texto en el segundo input
        input_text_search_child = self.obtenerXpath('//*[@id="empathy-x"]/header/div[1]/div/input[3]')
        input_text_search_child.send_keys(self.product_name)

        #Clickamos al button de buscar de la página
        button_submit = self.obtenerXpath('//*[@id="empathy-x"]/header/div/button[1]')
        button_submit.click()

        resultado = ''


        #Si el producto que buscamos no se encuentra en la pagina nos saltará una excepción
        #ya que estamos cogiendo el primer artículo que sale de la búsqueda
        try:
            # Vamos a los productos

            name = self.obtenerXpath('//*[@id="ebx-grid"]/article[1]/div/div[3]/a/h1').text
            price = self.obtenerXpath('//*[@id="ebx-grid"]/article[1]/div/p/strong')        
            resultado = price.text[: -1].replace(' ', '')
        except:
            name = self.product_name[0]
            resultado = "NO EXISTE EL PRODUCTO"


        return [name,resultado]


    #  Función que buscara el producto pasado a la clase en la web del AHORRA MAS
    def AhorraMasProduct(self):
        self.driver.get('https://www.ahorramas.com/')

        # Verificara si hemos aceptado las cookies anteriormente de esta página
        if not self.cookies:
            btn_cookies = self.obtenerXpath('//*[@id="onetrust-accept-btn-handler"]')
            btn_cookies.click()

        # Introducimos el texto en el input
        input_text = self.obtenerClassName('search-field')
        input_text.click()
        input_text.send_keys(self.product_name)
        
        #Clickamos al button de buscar de la página
        search_button = self.obtenerClassName('fa-search')
        search_button.click()


        resultado = ''

        #Si el producto que buscamos no se encuentra en la pagina nos saltará una excepción
        #ya que estamos cogiendo el primer artículo que sale de la búsqueda
        try:
            name = self.obtenerClassName('product-name-gtm').text
            price = self.obtenerXpath('//*[@id="product-search-results"]/div[2]/div[4]/div[2]/div[1]/div/div/div[3]/div[2]/div[1]/div/span/span')
            resultado = price.text[: -1].replace(' ', '')
        except:
            name = self.product_name[0]
            resultado = "NO EXISTE EL PRODUCTO"

        return [name,resultado]


    #  Función que buscara el producto pasado a la clase en la web del DÍA
    def DiaProduct(self):
        self.driver.get('https://www.dia.es/compra-online/')

        # Verificara si hemos aceptado las cookies anteriormente de esta página

        
        if not self.cookies:
            btn_cookie = self.obtenerXpath('//*[@id="onetrust-accept-btn-handler"]')
            time.sleep(2)
            btn_cookie.click()


        # Introducimos el texto en el input
        search_input = self.obtenerXpath('//*[@id="search"]')
        search_input.send_keys(self.product_name)

        #Clickamos al button de buscar de la página
        search_button = self.obtenerClassName('desktop-search')
        search_button.click()


        resultado = ''

        #Comprueba si hemos encontrado algun producto con el mismo nombre introducido
        #Si no existe dicho producto causara una excepcion que controlaremos y trataremos
        #Con la excepcion sabremos que en dicho supermercado no existe ningun producto con el 
        #nombre introducido
        try:
            name = self.obtenerXpath('//*[@id="productgridcontainer"]/div[1]/div[1]/div/a/div[2]/span').text
            price = self.obtenerXpath('//*[@id="productgridcontainer"]/div[1]/div[1]/div/a/div[2]/div/p[1]')
            resultado = price.text[: -1].replace(' ', '')
        except:
            name = self.product_name[0]
            resultado = 'NO EXISTE EL PRODUCTO'

        return [name, resultado]


    # Busca el producto en distintos supermercados
    def BuscarProductos(self):
        Carrefour = self.CarrefourProduct()
        AhorraMas = self.AhorraMasProduct()
        Dia = self.DiaProduct()
        return [Carrefour,AhorraMas,Dia]


    def obtenerXpath(self, xPath:str):
        return self.wait1.until(EC.presence_of_element_located((
                By.XPATH, xPath)))

    def obtenerClassName(self, ClassName):
        return self.wait1.until(EC.presence_of_element_located((
                By.CLASS_NAME, ClassName)))


#### BUSCA PRODUCTOS Y GUARDA EN EXCEL


In [10]:
#Hacemos carga del archivo csv
listaProductos = cargarCSV('entradas/lista_compra.csv')

#Abrimos la pagina web
driver = webdriver.Chrome()

#La maximizamos
driver.maximize_window()

#Empezamos a buscar los productos de uno en uno
listaPrecios = []
cookies:bool = False

for product in listaProductos:
    search = Search(product_name=product, driver=driver, cookies=cookies)
    listaPrecios.append(search.BuscarProductos())
    cookies = True


GuardarExcel("salidas/products.xlsx", listaProductos, listaPrecios)

##### Guardar Productos Buscados en una Base de datos

In [11]:
# Establecer la conexión a la base de datos
conexion = mysql.connector.connect(
    host='127.0.0.1',
    user='prueba',
    password='1234',
    database='comparador_precios'
)

# Crear un objeto cursor
cursor = conexion.cursor()

consulta = "INSERT INTO PRODUCTO (NOMBRE, PRECIO, ULTIMA_ACTUALIZACION, ID_SUPERMERCADO) VALUES (%s, %s, %s, %s)"

lista = []

for producto in listaPrecios:
    #Valores del array para los precios del CARREFOUR
    nombreProductoCarrefour = producto[0][0]

    if producto[0][1] == 'NO EXISTE EL PRODUCTO':
        precioProductoCarrefour = None
    else:
        precioProductoCarrefour = float(producto[0][1].replace(",",".")) 

    #Valores del array para los precios del AHORRA MAS
    nombreProductoAhorraMas = producto[1][0]

    if producto[1][1] == 'NO EXISTE EL PRODUCTO':
        precioProductoAhorraMas = None
    else:
        precioProductoAhorraMas = float(producto[1][1].replace(",",".")) 

    #Valores del array para los precios del DIA
    nombreProductoDia = producto[2][0]
    if producto[2][1] == 'NO EXISTE EL PRODUCTO':
        precioProductoDia = None
    else:
        precioProductoDia = float(producto[2][1].replace(",",".")) 
    

    #Fecha actual
    fecha_actual = datetime.datetime.now()
    timestamp = fecha_actual.strftime('%Y-%m-%d %H:%M:%S')

    #Obtenemos los valores de cada producto para insertarlos ahora despues
    lista.append((nombreProductoCarrefour, precioProductoCarrefour, timestamp, 1))
    lista.append((nombreProductoAhorraMas, precioProductoAhorraMas, timestamp, 2))
    lista.append((nombreProductoDia, precioProductoDia, timestamp, 3))

for valor in lista:
    #Ejecutamos todos los inserts con sus valores
    cursor.execute(consulta, valor)

# Hacer commit de la transacción
conexion.commit()

# Cerrar la conexión
conexion.close()